In [1]:
import matplotlib.pyplot as plt
from collections import Counter
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import numpy as np
import re
import datetime
from pylab import rcParams
import seaborn as sns
from collections import Counter
sns.set(rc={'figure.figsize':(11, 10)})

pd.set_option('display.max_rows', 500)

In [2]:
Arabic_folder = 'Arabic'
English_folder = 'English'
Spanish_folder = 'Spanish'
French_folder = 'French'

In [61]:

def clean_spaces(data):
    df_obj = data.select_dtypes(['object'])
    data[df_obj.columns] = df_obj.apply(lambda x: x.str.strip()).values
    return data

    
    
def clean_ar_df (data , cleaning):
    data['Suggested Themes'] = data['Suggested Themes'].replace(cleaning, regex=True)
    extracted = data['Suggested Themes'].apply(lambda x: re.split('; |,|\*|\n',str(x))).explode()
    extracted = extracted.str.strip().value_counts().to_frame().reset_index()
    cleaned = extracted[extracted[extracted.columns[0]].str.strip().astype(bool)]
    cleaned = cleaned[cleaned['index'].str.strip('nan').astype(bool)]
    cleaned = cleaned.rename(columns={'index': 'Suggested Themes', 'Suggested Themes': 'Count'})
    return cleaned

    

    
def clean_en_df (data , cleaning):
#    
#     if  'china-US' in list(data['Suggested Themes']):
#         data['Suggested Themes'] = data['Suggested Themes'].replace({'china-US' : 'China-US relations'}, regex=True)
#     "China's relations with Africa" : "China's relations with African countries" ,
#     if  'China and the neighbours' in list(data['Suggested Themes']):
#         data['Suggested Themes'] = data['Suggested Themes'].replace({'China and the neighbours' : "China’s relations with neighbours"}, regex=True)

#     if  "China's relations with Africa" in list(data['Suggested Themes']):
#         data['Suggested Themes'] = data['Suggested Themes'].replace({"China's relations with Africa" : "China's relations with African countries"}, regex=True)
#             'China and the neighbours' : "China’s relations with neighbours" ,

    

    data['Suggested Themes'] = data['Suggested Themes'].replace(cleaning, regex=True)
    extracted = data['Suggested Themes'].apply(lambda x: re.split('; |, |\*|\n',str(x))).explode()
    extracted = extracted.str.strip().value_counts().to_frame().reset_index()
    cleaned = extracted[extracted[extracted.columns[0]].str.strip().astype(bool)]
    cleaned = cleaned[cleaned['index'].str.strip('nan').astype(bool)]
    cleaned = cleaned[cleaned['index'].str.strip(',').astype(bool)]
    cleaned = cleaned.rename(columns={'index': 'Suggested Themes', 'Suggested Themes': 'Count'})

#     data['Suggested Themes'] = data['Suggested Themes'].str.strip()
#     extracted = data['Suggested Themes'].apply(lambda x: re.split('; |, |\*|\n',str(x))).explode()
#     extracted.str.strip().value_counts()

#     extracted = data['Suggested Themes'].apply(lambda x: re.split('; |, |\*|\n',str(x))).explode().value_counts().reset_index()
#     cleaned = extracted[extracted['index'].str.strip().astype(bool)]
#     cleaned = extracted[extracted['index'].str.strip(['nan',]).astype(bool)]
#     cleaned = cleaned.rename(columns={'index': 'Suggested Themes', 'Suggested Themes': 'Count'})
    return cleaned

    


def clean_es_df (data , cleaning):
    data['Suggested Themes'] = data['Suggested Themes'].str.strip()
    data['Suggested Themes'] = data['Suggested Themes'].replace(cleaning, regex=True)
    if 'Chinese Media coverage of foreign countries' in list(data['Suggested Themes']):
        data['Suggested Themes'] = data['Suggested Themes'].replace({'Chinese Media coverage of foreign countries' : 'Foreign news'}, regex=True)    
    if 'Belt and Road initiative (BRI)' in list(data['Suggested Themes']):
        data['Suggested Themes'] = data['Suggested Themes'].replace({'Belt and Road initiative (BRI)' : 'Belt and Road Initiative (BRI)'}, regex=True)    
        
    
   
    if 'Development/Development/aid' in list(data['Suggested Themes']):
            data['Suggested Themes'] = data['Suggested Themes'].replace({'Development/Development/aid' : 'Development/aid'}, regex=True)
    
    extracted = data['Suggested Themes'].apply(lambda x: re.split('; |, |\*|\n',str(x))).explode().value_counts().reset_index()
    cleaned = extracted[extracted['index'].str.strip().astype(bool)]
    cleaned = extracted[extracted['index'].str.strip('nan').astype(bool)]
    cleaned = cleaned.rename(columns={'index': 'Suggested Themes', 'Suggested Themes': 'Count'})
    return cleaned


def clean_fr_df (data , cleaning):
        
        if 'Environment' in list(data['Suggested Themes']): 
            data['Suggested Themes'] = data['Suggested Themes'].replace({'Environment' : 'Environmental protection'}, regex=True)
        
        data['Suggested Themes'] = data['Suggested Themes'].replace(cleaning, regex=True)
        
        data['Suggested Themes'] = data['Suggested Themes'].replace({'Environmental protectional protection' : 'Environmental protection'}, regex=True)

        if 'Development' in list(data['Suggested Themes']): 
            data['Suggested Themes'] = data['Suggested Themes'].replace({'Development' : 'Development/aid'}, regex=True)
            
        extracted = data['Suggested Themes'].apply(lambda x: re.split('; |,|\*|\n',str(x))).explode()
        extracted = extracted.str.strip().value_counts().to_frame().reset_index()
        cleaned = extracted[extracted[extracted.columns[0]].str.strip().astype(bool)]
        cleaned = cleaned[cleaned['index'].str.strip('nan').astype(bool)]
        cleaned = cleaned.rename(columns={'index': 'Suggested Themes', 'Suggested Themes': 'Count'})
        return cleaned


def remove_spaces_from_df (data):
        data['Suggested Themes'] = data['Suggested Themes'].str.strip()
        extracted = data['Suggested Themes'].apply(lambda x: re.split('; |, |\*|\n',str(x))).explode().value_counts().reset_index()
        cleaned = extracted.rename(columns={'index': 'Suggested Themes', 'Suggested Themes': 'Count'})
        return cleaned
    
    
def combine_tw_fb_counts(tw,fb):
    combined_tw_fb_themes_counts = tw.set_index('Suggested Themes').join(fb.set_index('Suggested Themes'), lsuffix='_Of_Twitter', rsuffix='_Of_Facebook').fillna(0)
    combined_tw_fb_themes_counts['Count_Of_Facebook'] = combined_tw_fb_themes_counts['Count_Of_Facebook'].astype(int)
    Total = combined_tw_fb_themes_counts['Count_Of_Twitter'] + combined_tw_fb_themes_counts['Count_Of_Facebook']
    combined_tw_fb_themes_counts['Total'] = Total
    Final = combined_tw_fb_themes_counts.reset_index().sort_values('Total', ascending= False)
    return Final
                     




### Arabic Team Preformance 

In [13]:
ar_fb = pd.read_excel(Arabic_folder+'/ar-fb-random-500-28-Oct.xlsx')
ar_tw = pd.read_excel(Arabic_folder+'/ar-tw-random-500-28-Oct.xlsx')

In [14]:
ar_cleaning_dict = {
    'Covid-19 diplomacy' : 'Covid-19 Diplomacy' ,
    'the Chinese political system' : 'Chinese political system' ,
}

In [77]:
ar_fb_themes_count = clean_ar_df(ar_fb,ar_cleaning_dict)
ar_tw_themes_count = clean_ar_df(ar_tw , ar_cleaning_dict)
combine_ar_tw_fb_counts = combine_tw_fb_counts(ar_tw_themes_count,ar_fb_themes_count)
combine_ar_tw_fb_counts.to_csv('Arabic/reports/Arabic_extracted_themes.csv',index = False)


In [22]:
print(f" The number of annotated facebook posts: {ar_fb['Suggested Themes'].value_counts().reset_index()['Suggested Themes'].sum()}")
print(f" The number of annotated tweets : {ar_tw['Suggested Themes'].value_counts().reset_index()['Suggested Themes'].sum()}")


 The number of annotated facebook posts: 250
 The number of annotated tweets : 480


### English Team Preformance


In [23]:
en_fb = pd.read_excel(English_folder+'/en-fb-random-500-28-Oct.xlsx')
en_tw = pd.read_excel(English_folder+'/en-tw-random-500-28-Oct.xlsx')


In [24]:
en_cleaning_dict = {
    'xi jinping' : 'Xi Jinping' ,
    'health' : 'Health' , 
    'Covid-19 diplomacy' : 'Covid-19 Diplomacy' ,
    'china and the world' : 'China and the World' ,
    'covid-19' : 'Covid-19' ,
    'to be omitted?' : 'To be omitted' ,
    'Chinese people and culture' : 'Chinese culture and people' ,
    'chinese people and culture' : 'Chinese culture and people' ,
    'China and the world' : 'China and the World' ,
    'chinese people' : 'Chinese culture and people' ,
    'Chinese people' : 'Chinese culture and people' ,
    'diplomacy' : 'Diplomacy' ,
    'Food diplomacy' : 'Food Diplomacy' ,
    'development/aid' : 'Development/aid' ,
    'cooperation' : 'Cooperation' ,
    'politics' : 'Politics' ,
    'economy' : 'Economy ' ,
    'anti-US' : 'Anti-US' ,
    'chinese leadership appearances and messaging' : 'Chinese leadership appearances and messaging' ,
    'technology' : 'Technology' ,
    'the Chinese political system' : 'Chinese political system' ,
#     'china and the UN' : 'China’s relations with the UN' ,
    'Chinese leadership apperaances and messaging' : 'Chinese leadership appearances and messaging' ,
    'Turming messages back on US/West' : 'Turning messages back on US/West' ,
    'china\'s relations with african countries' : 'China\'s relations with African countries' ,
    'china\'s relations with african countries' : 'China\'s relations with African countries' ,
    'china\'s relations with Africa' : 'China\'s relations with African countries' ,
    'countering negative news on china' : 'Countering negative news on China' ,
    'anti-Trump' : 'Anti-Trump' ,
    'US-China' : 'China-US relations' ,
#     'China and the neighbours' : "China’s relations with neighbours" ,
    "China’s relations with the US" : 'China-US relations' ,
#     'china and UN' : 'China’s relations with the UN' ,
    'chinese leadership messaging and apperances' : 'Chinese leadership appearances and messaging' ,
#     'Military modernisation?' : 'Modernisation' ,
#     'china-US' : 'China-US relations' ,
#     'China-US' : 'China-US relations' ,
#     'china and the EU' : 'China’s relations with the EU' ,
    'China and the WRold' : 'China and the World' ,
#     "China's relations with Africa" : "China's relations with African countries" ,
    'foreign media coverage of China' : 'Foreign media coverage of China' ,
    'South China' : 'South China Sea' ,
#     'china and EU' : 'China’s relations with the EU' ,
    'education' : 'Education' ,
    '(China and US)' : 'China-US relations' ,
    'chinese leadership apperances and messaging' : 'Chinese leadership appearances and messaging' ,
    'development/ aid' : 'Development/aid' ,
#     'China's relation with the UN' : 'China’s relations with the UN' ,
    'treatment of minorities in US' : 'Treatment of minorities in US' ,
    'Historical injustices against China' : 'Historical injustices towards China' ,
    'China\'s allies' : 'Allies' ,
    'freedom of speech' : 'Freedom of speech'
}

In [25]:
en_fb_themes_count = clean_en_df(en_fb,en_cleaning_dict)
# Delete the extra China's relations with the UN
index = en_fb_themes_count.loc[(en_fb_themes_count['Suggested Themes'] == 'China’s relations with the UN') | (en_fb_themes_count['Count'] == '1')].index[0]
en_fb_themes_count = en_fb_themes_count.drop(index) 
en_tw_themes_count = clean_en_df(en_tw,en_cleaning_dict)

combine_en_tw_fb_counts = combine_tw_fb_counts(en_tw_themes_count,en_fb_themes_count)


/Users/ay227/Desktop/first-trial/FT3.6/lib/python3.6/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)


In [26]:
combine_en_tw_fb_counts.to_csv('English/reports/English_extracted_themes.csv',index = False)


In [27]:
print(f" The number of annotated facebook posts: {en_fb['Suggested Themes'].value_counts().reset_index()['Suggested Themes'].sum()}")
print(f" The number of annotated tweets : {en_tw['Suggested Themes'].value_counts().reset_index()['Suggested Themes'].sum()}")


 The number of annotated facebook posts: 477
 The number of annotated tweets : 399


### Spanish Team Preformance 

In [28]:
es_cleaning_dict = {'cooperation' : 'Cooperation' ,
                     'coorperation' : 'Cooperation' ,
                     'development/aid' : 'Development/aid' ,
                     'economy' : 'Economy' ,
                     'health' : 'Health' , 
                     'countering negative news on China' : 'Countering negative news on China' , 
                     'Countering negative news of China' : 'Countering negative news on China' , 
                     'countering negative news of China' : 'Countering negative news on China' ,
                     'Aid' : 'Development/aid' , 
                     'media' : 'Media' ,
                     'Chinese leadership appearances and messaging to the world' : 'Chinese leadership appearances and messaging' , 
                     'tourism': 'Tourism' ,
                     'technology' : 'Technology' ,
                     'BLM' : 'Black Lives Matter' ,
                     'allies' : 'Allies' , 
                     'politics' : 'Politics' ,
                     'diplomacy' : 'Diplomacy' , 
                     'aid' : 'Development/aid' ,
                     'environmental protection' : 'Environmental protection' , 
                     'poverty alleviation' : 'Poverty alleviation' , 
                     'covid-19' : 'Covid-19' , 
                     'economic reform' : 'Economic reform' , 
                     'turning messages back on US' : 'Turning messages back on US' ,
                     'trade' : 'Trade' , 
                     'modernisation' : 'Modernisation' , 
                     'education' : 'Education' , 
                     'Environemental protection' : 'Environmental protection' ,
                     'infrastructure' : 'Infrastructure' ,
                     'Enviornmental protection' : 'Environmental protection' ,
                     'green energy' : 'Green energy' ,
                     'history' : 'Chinese history' ,
                     'Xi Jingping' : 'Xi Jinping' , 
                     'human rights' : 'Human rights' ,
                     'irrelevant' : 'Irrelevant' ,
                     'Chinese culutre and people' : 'Chinese culture and people'
                    }

In [41]:
initial_es_tw = pd.read_excel(Spanish_folder+'/es-initial-sampe-1000-27-10-finished_work_on_the_other_files.xlsx',converters={'Suggested Themes': str.lstrip})
es_tw = pd.read_excel(Spanish_folder+'/es-tw-random-500-28-Oct-annotate-only-300.xlsx')
combined_es_tw = pd.concat([initial_es_tw,es_tw]) 
es_fb = pd.read_excel(Spanish_folder+'/es-fb-random-500-28-Oct.xlsx')


In [42]:
es_tw_themes_count = clean_es_df(combined_es_tw , es_cleaning_dict)
es_fb_themes_count = clean_es_df(es_fb , es_cleaning_dict)
combine_es_tw_fb_counts = combine_tw_fb_counts(es_tw_themes_count,es_fb_themes_count)


In [45]:
combine_es_tw_fb_counts.to_csv('Spanish/reports/Spanish_extracted_themes.csv',index = False)


In [44]:
print(f" The number of annotated facebook posts: {es_fb['Suggested Themes'].value_counts().reset_index()['Suggested Themes'].sum()}")
print(f" The total number of annotated tweets : {es_tw['Suggested Themes'].value_counts().reset_index()['Suggested Themes'].sum() + initial_es_tw['Suggested Themes'].value_counts().reset_index()['Suggested Themes'].sum()}")
print(f" The number of annotated tweets : {es_tw['Suggested Themes'].value_counts().reset_index()['Suggested Themes'].sum()}")
print(f" The number of initial annotated tweets : {initial_es_tw['Suggested Themes'].value_counts().reset_index()['Suggested Themes'].sum()}")


 The number of annotated facebook posts: 496
 The total number of annotated tweets : 531
 The number of annotated tweets : 298
 The number of initial annotated tweets : 233


### French Team Preformance 

In [62]:
fr_cleaning_dict = {
    'green technology' : 'Green energy' , 
    'sustainable development' : ' Sustainable Development' ,
    'virus' : 'Covid-19' ,
    'trade' : 'Trade' ,
    'Chinese people and culture' : 'Chinese culture and people' ,
    'tourism' : 'Tourism' ,
    'economy' : 'Economy' ,
    'food diplomacy' : 'Food Diplomacy' ,
    'Chinese relations with Arab countries' : 'China\'s relations with Arab countries' ,
    'foreign media coverage of China' : 'Foreign media coverage of China' ,
    'cultural diplomacy' : 'Cultural Diplomacy' ,
    'technology' : 'Technology' ,
#     'development' : 'Development/aid' ,
    'irrelevant' : 'Irrelevant' ,
    'Chinese realtions with Arab countries' : 'China\'s relations with Arab countries' ,
    'Chinese Economic Development' : 'Chinese economic development' ,
    'Covid-10' : 'Covid-19' ,
#     'development aid' : 'Development/aid' ,
    'Chinese relations with the World' : 'China and the World' ,
    'China\'s relations with Afrrican countries' : 'China\'s relations with African countries' ,
    'Chinese relations with African countries' : 'China\'s relations with African countries' ,
    'health' : 'Health' ,
#     'Development' : 'Development/aid' , 
    'China relations with Africa' : 'China\'s relations with African countries' ,
#     'Environment' : 'Environmental protection' ,
    'COVID-19' : 'Covid-19' ,
    'Chinese media' : 'Foreign news' ,
    'poverty alleviation' : 'Poverty alleviation' ,
    'diplomacy' : 'Diplomacy' , 
    
    'Hongkong' : 'Hong Kong' ,
    'China\'s relations with Africa' : 'China\'s relations with African countries' ,
    'China relations with UN' : 'China\'s relations with the UN' ,
    'China relations and the world' : 'China and the World' ,
    'politics' : 'Politics' , 
    'wildlife' : 'Wildlife' ,
    'chinese festivals' : 'Chinese festivals' ,
    'education' : 'Education' ,
    'Chinese relations with the EU' : 'China’s relations with the EU'
   
}

In [68]:
fr_fb = pd.read_excel(French_folder+'/fr-fb-random-500-28-Oct.xlsx')
fr_tw = pd.read_excel(French_folder+'/fr-tw-random-500-28-Oct.xlsx')

In [72]:
fr_fb_themes_count = clean_fr_df(fr_fb , fr_cleaning_dict)
fr_tw_themes_count = clean_fr_df(fr_tw , fr_cleaning_dict)
combine_es_tw_fb_counts = combine_tw_fb_counts(fr_fb_themes_count,fr_tw_themes_count)


In [74]:
# This is to remove something i am not sure about it 
combine_es_tw_fb_counts = combine_es_tw_fb_counts.drop(29)

In [75]:
combine_es_tw_fb_counts.to_csv('French/reports/French_extracted_themes.csv',index = False)


In [76]:
print(f" The number of annotated facebook posts: {fr_fb['Suggested Themes'].value_counts().reset_index()['Suggested Themes'].sum()}")
print(f" The number of annotated tweets : {fr_tw['Suggested Themes'].value_counts().reset_index()['Suggested Themes'].sum()}")


 The number of annotated facebook posts: 102
 The number of annotated tweets : 135
